# This notebook creates Odds for every match in NBA season 2023/24

In [3]:
import pandas as pd
from unidecode import unidecode
from tqdm import tqdm
import numpy as np
from dateutil.parser import parse
pd.options.display.max_colwidth = 150
pd.set_option('display.max_columns', None)

# Create a DF that will contain the odds

## For every match in schedule we have to connect all players on both teams roster

In [48]:
schedule = pd.read_csv("schedule_trans.csv")
schedule.head()

,Link,Date,Start,Visitor,Visitor Link,Visitor PTS,Home,Home Link,Home PTS,Match Link,Overtimes,Attendance,Game Length,Arena,Notes
0,https://www.basketball-reference.com/leagues/NBA_2024_games-october.html,"Tue, Oct 24, 2023",7:30p,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,107,Denver Nuggets,https://www.basketball-reference.com/teams/DEN/2024.html,119,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,"19,842",2:17,Ball Arena,NaN
1,https://www.basketball-reference.com/leagues/NBA_2024_games-october.html,"Tue, Oct 24, 2023",10:00p,Phoenix Suns,https://www.basketball-reference.com/teams/PHO/2024.html,108,Golden State Warriors,https://www.basketball-reference.com/teams/GSW/2024.html,104,https://www.basketball-reference.com/boxscores/202310240GSW.html,NaN,"18,064",2:33,Chase Center,NaN
2,https://www.basketball-reference.com/leagues/NBA_2024_games-october.html,"Wed, Oct 25, 2023",7:00p,Houston Rockets,https://www.basketball-reference.com/teams/HOU/2024.html,86,Orlando Magic,https://www.basketball-reference.com/teams/ORL/2024.html,116,https://www.basketball-reference.com/boxscores/202310250ORL.html,NaN,"18,846",2:14,Kia Center,NaN
3,https://www.basketball-reference.com/leagues/NBA_2024_games-october.html,"Wed, Oct 25, 2023",7:00p,Boston Celtics,https://www.basketball-reference.com/teams/BOS/2024.html,108,New York Knicks,https://www.basketball-reference.com/teams/NYK/2024.html,104,https://www.basketball-reference.com/boxscores/202310250NYK.html,NaN,"19,812",2:39,Madison Square Garden (IV),NaN
4,https://www.basketball-reference.com/leagues/NBA_2024_games-october.html,"Wed, Oct 25, 2023",7:00p,Washington Wizards,https://www.basketball-reference.com/teams/WAS/2024.html,120,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,143,https://www.basketball-reference.com/boxscores/202310250IND.html,NaN,"16,004",2:07,Gainbridge Fieldhouse,NaN


In [49]:
rosters = pd.read_csv("rosters_trans.csv")
rosters.head()

,Team,Team Link,Number,Player,Player Link,Position
0,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,11,Bruce Brown,https://www.basketball-reference.com/players/b/brownbr01.html,SG
1,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,10,Kendall Brown,https://www.basketball-reference.com/players/b/brownke03.html,SG
2,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,0,Tyrese Haliburton,https://www.basketball-reference.com/players/h/halibty01.html,PG
3,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,7,Buddy Hield,https://www.basketball-reference.com/players/h/hieldbu01.html,SF
4,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,22,Isaiah Jackson,https://www.basketball-reference.com/players/j/jacksis01.html,C


In [50]:
# Select only usefull columns
df1 = schedule[["Date", "Visitor", "Visitor PTS", "Home", "Home PTS", "Visitor Link", "Home Link", "Match Link", "Overtimes"]]
df1.head()

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN
1,"Tue, Oct 24, 2023",Phoenix Suns,108,Golden State Warriors,104,https://www.basketball-reference.com/teams/PHO/2024.html,https://www.basketball-reference.com/teams/GSW/2024.html,https://www.basketball-reference.com/boxscores/202310240GSW.html,NaN
2,"Wed, Oct 25, 2023",Houston Rockets,86,Orlando Magic,116,https://www.basketball-reference.com/teams/HOU/2024.html,https://www.basketball-reference.com/teams/ORL/2024.html,https://www.basketball-reference.com/boxscores/202310250ORL.html,NaN
3,"Wed, Oct 25, 2023",Boston Celtics,108,New York Knicks,104,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/teams/NYK/2024.html,https://www.basketball-reference.com/boxscores/202310250NYK.html,NaN
4,"Wed, Oct 25, 2023",Washington Wizards,120,Indiana Pacers,143,https://www.basketball-reference.com/teams/WAS/2024.html,https://www.basketball-reference.com/teams/IND/2024.html,https://www.basketball-reference.com/boxscores/202310250IND.html,NaN


In [51]:
# Select only usefull columns
df2 = rosters[["Team", "Team Link", "Player", "Number", "Player Link"]]
df2.head()

,Team,Team Link,Player,Number,Player Link
0,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Bruce Brown,11,https://www.basketball-reference.com/players/b/brownbr01.html
1,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Kendall Brown,10,https://www.basketball-reference.com/players/b/brownke03.html
2,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Tyrese Haliburton,0,https://www.basketball-reference.com/players/h/halibty01.html
3,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Buddy Hield,7,https://www.basketball-reference.com/players/h/hieldbu01.html
4,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Isaiah Jackson,22,https://www.basketball-reference.com/players/j/jacksis01.html


### Merge

We cross merge Schedule with Rosters to connect all Players from Rosters to every single match in Schedule.

In [52]:
odds = pd.merge(df1, df2, how="cross")
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Bruce Brown,11,https://www.basketball-reference.com/players/b/brownbr01.html
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Kendall Brown,10,https://www.basketball-reference.com/players/b/brownke03.html
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Tyrese Haliburton,0,https://www.basketball-reference.com/players/h/halibty01.html
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Buddy Hield,7,https://www.basketball-reference.com/players/h/hieldbu01.html
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Indiana Pacers,https://www.basketball-reference.com/teams/IND/2024.html,Isaiah Jackson,22,https://www.basketball-reference.com/players/j/jacksis01.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866578,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Memphis Grizzlies,https://www.basketball-reference.com/teams/MEM/2024.html,Xavier Tillman Sr.,2,https://www.basketball-reference.com/players/t/tillmxa01.html
866579,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Memphis Grizzlies,https://www.basketball-reference.com/teams/MEM/2024.html,Yuta Watanabe,18,https://www.basketball-reference.com/players/w/watanyu01.html
866580,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Memphis Grizzlies,https://www.basketball-reference.com/teams/MEM/2024.html,Jack White,14,https://www.basketball-reference.com/players/w/whiteja03.html
866581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Memphis Grizzlies,https://www.basketball-reference.com/teams/MEM/2024.html,Vince Williams Jr.,5,https://www.basketball-reference.com/players/w/willivi01.html


We only keep these rows where Player is either from a Visitors team or Home team.

In [53]:
odds = odds[(odds['Visitor Link'] == odds['Team Link']) | (odds['Home Link'] == odds['Team Link'])].reset_index(drop=True)
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Olivier-Maxence Prosper,18,https://www.basketball-reference.com/players/p/prospol01.html
57583,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,P.J. Washington,25,https://www.basketball-reference.com/players/w/washipj01.html
57584,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Brandon Williams,00,https://www.basketball-reference.com/players/w/willibr03.html


In [54]:
schedule.shape

(1319, 15)

In [55]:
rosters.shape

(657, 6)

**Each team has about 20 players and schedule has about 1300 matches if we multiply this (2 times 20 times 1300) we get 52 000 which is close to the number of rows we got.**

#### Create a column Side to determine whether a player belongs to visitor or home team

In [56]:
odds["Side"] = np.where(odds['Visitor Link'] == odds['Team Link'], 'Visitor', 'Home')
odds["Side"].value_counts()

Side
Visitor    28802
Home       28784
Name: count, dtype: int64

In [57]:
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html,Visitor
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Olivier-Maxence Prosper,18,https://www.basketball-reference.com/players/p/prospol01.html,Visitor
57583,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,P.J. Washington,25,https://www.basketball-reference.com/players/w/washipj01.html,Visitor
57584,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Brandon Williams,00,https://www.basketball-reference.com/players/w/willibr03.html,Visitor


# Calculate the Average for every player

In [58]:
splits = pd.read_csv("splits_trans.csv")
splits.head()

,Name,Link,Year,Split,Value,G,GS,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,TS%,USG%,ORtg,DRtg,+/-
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Total,Total,74,18,1624,235,469,26,97,69,112,191,487,97,46,68,83,143,565,0.501,0.268,0.616,0.545,16.2,113,112,-1.5
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Place,Home,44,13,1038,156,314,14,63,44,68,121,310,57,29,50,49,80,370,0.497,0.222,0.647,0.538,16.5,113,111,-0.9
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Place,Road,30,5,585,79,155,12,34,25,44,70,177,40,17,18,34,63,195,0.510,0.353,0.568,0.559,15.7,113,112,-2.4
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,All-Star,Pre,48,9,1020,158,322,19,71,41,64,124,303,65,29,36,50,88,376,0.491,0.268,0.641,0.537,16.9,113,114,-0.2
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,All-Star,Post,26,9,604,77,147,7,26,28,48,67,184,32,17,32,33,55,189,0.524,0.269,0.583,0.562,15.0,112,108,-3.7


### Update columns to give weight 3 to 2024, weight 2 to 2023 and weight 1 to all other seasons

In [59]:
# Update 'G' column
splits.loc[splits['Year'] == 2024, 'G'] *= 3
splits.loc[splits['Year'] == 2023, 'G'] *= 2

# Update 'MP' column
splits.loc[splits['Year'] == 2024, 'MP'] *= 3
splits.loc[splits['Year'] == 2023, 'MP'] *= 2

# Update '3P' column
splits.loc[splits['Year'] == 2024, '3P'] *= 3
splits.loc[splits['Year'] == 2023, '3P'] *= 2

# Update 'STL' column
splits.loc[splits['Year'] == 2024, 'STL'] *= 3
splits.loc[splits['Year'] == 2023, 'STL'] *= 2

# Update 'BLK' column
splits.loc[splits['Year'] == 2024, 'BLK'] *= 3
splits.loc[splits['Year'] == 2023, 'BLK'] *= 2

# Update 'TOV' column
splits.loc[splits['Year'] == 2024, 'TOV'] *= 3
splits.loc[splits['Year'] == 2023, 'TOV'] *= 2

In [60]:
splits

,Name,Link,Year,Split,Value,G,GS,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,TS%,USG%,ORtg,DRtg,+/-
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Total,Total,222,18,4872,235,469,78,97,69,112,191,487,97,138,204,249,143,565,0.501,0.268,0.616,0.545,16.2,113,112,-1.5
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Place,Home,132,13,3114,156,314,42,63,44,68,121,310,57,87,150,147,80,370,0.497,0.222,0.647,0.538,16.5,113,111,-0.9
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Place,Road,90,5,1755,79,155,36,34,25,44,70,177,40,51,54,102,63,195,0.510,0.353,0.568,0.559,15.7,113,112,-2.4
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,All-Star,Pre,144,9,3060,158,322,57,71,41,64,124,303,65,87,108,150,88,376,0.491,0.268,0.641,0.537,16.9,113,114,-0.2
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,All-Star,Post,78,9,1812,77,147,21,26,28,48,67,184,32,51,96,99,55,189,0.524,0.269,0.583,0.562,15.0,112,108,-3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104835,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2022,Opponent,Sacramento,4,4,104,23,31,0,0,8,10,11,36,5,2,3,3,10,54,0.742,0.000,0.800,0.763,15.7,149,103,4.3
104836,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2022,Opponent,San Antonio,3,3,79,11,21,0,0,4,6,14,34,4,2,3,3,7,26,0.524,0.000,0.667,0.550,14.3,120,98,-13.1
104837,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2022,Opponent,Toronto,1,1,17,1,4,0,0,0,0,1,7,2,1,0,3,1,2,0.250,0.000,0.000,0.250,20.0,53,105,-40.6
104838,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2022,Opponent,Utah,3,3,56,7,17,0,0,7,7,9,18,1,3,2,2,9,21,0.412,0.000,1.000,0.523,17.7,117,121,-44.7


### Keep data only for Total, Place, Result and Opponent splits

In [61]:
splits = splits.loc[splits["Split"].isin(["Total", "Place", "Result", "Opponent"])]
splits.head()

,Name,Link,Year,Split,Value,G,GS,MP,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,TS%,USG%,ORtg,DRtg,+/-
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Total,Total,222,18,4872,235,469,78,97,69,112,191,487,97,138,204,249,143,565,0.501,0.268,0.616,0.545,16.2,113,112,-1.5
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Place,Home,132,13,3114,156,314,42,63,44,68,121,310,57,87,150,147,80,370,0.497,0.222,0.647,0.538,16.5,113,111,-0.9
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Place,Road,90,5,1755,79,155,36,34,25,44,70,177,40,51,54,102,63,195,0.510,0.353,0.568,0.559,15.7,113,112,-2.4
5,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Result,Win,120,8,2454,125,243,36,40,28,49,105,261,47,84,120,126,67,290,0.514,0.300,0.571,0.548,16.4,115,103,10.8
6,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2024,Result,Loss,102,10,2418,110,226,42,57,41,63,86,226,50,54,84,123,76,275,0.487,0.246,0.651,0.542,16.0,110,120,-13.9


In [62]:
splits_grouped = splits.groupby(["Link", "Name", "Split", "Value"]).agg(
    G_sum = ("G", "sum"),
    MP_sum = ("MP", "sum"),
    Three_P_sum = ("3P", "sum"),
    STL_sum = ("STL", "sum"),
    BLK_sum = ("BLK", "sum"),
    TOV_sum = ("TOV", "sum"),
).reset_index()
splits_grouped

,Link,Name,Split,Value,G_sum,MP_sum,Three_P_sum,STL_sum,BLK_sum,TOV_sum
0,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Atlanta,17,354,8,10,10,15
1,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Boston,25,561,8,14,19,19
2,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Brooklyn,17,240,7,4,1,13
3,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Charlotte,20,398,25,18,6,26
4,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Chicago,25,444,4,6,20,30
...,...,...,...,...,...,...,...,...,...,...
22515,https://www.basketball-reference.com/players/z/zubaciv01.html,Ivica Zubac,Place,Home,217,5689,0,71,249,300
22516,https://www.basketball-reference.com/players/z/zubaciv01.html,Ivica Zubac,Place,Road,215,5887,0,89,273,285
22517,https://www.basketball-reference.com/players/z/zubaciv01.html,Ivica Zubac,Result,Loss,180,4761,0,54,194,247
22518,https://www.basketball-reference.com/players/z/zubaciv01.html,Ivica Zubac,Result,Win,252,6813,0,106,328,338


### Calculate stats per game

In [63]:
splits_grouped["MP_game"] = splits_grouped["MP_sum"] / splits_grouped["G_sum"]
splits_grouped["3P_game"] = splits_grouped["Three_P_sum"] / splits_grouped["G_sum"]
splits_grouped["STL_game"] = splits_grouped["STL_sum"] / splits_grouped["G_sum"]
splits_grouped["BLK_game"] = splits_grouped["BLK_sum"] / splits_grouped["G_sum"]
splits_grouped["TOV_game"] = splits_grouped["TOV_sum"] / splits_grouped["G_sum"]
splits_grouped.head()

,Link,Name,Split,Value,G_sum,MP_sum,Three_P_sum,STL_sum,BLK_sum,TOV_sum,MP_game,3P_game,STL_game,BLK_game,TOV_game
0,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Atlanta,17,354,8,10,10,15,20.823529,0.470588,0.588235,0.588235,0.882353
1,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Boston,25,561,8,14,19,19,22.440000,0.320000,0.560000,0.760000,0.760000
2,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Brooklyn,17,240,7,4,1,13,14.117647,0.411765,0.235294,0.058824,0.764706
3,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Charlotte,20,398,25,18,6,26,19.900000,1.250000,0.900000,0.300000,1.300000
4,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Chicago,25,444,4,6,20,30,17.760000,0.160000,0.240000,0.800000,1.200000


### Calculate stats per minute

In [64]:
splits_grouped["3P_min"] = splits_grouped["Three_P_sum"] / splits_grouped["MP_sum"]
splits_grouped["STL_min"] = splits_grouped["STL_sum"] / splits_grouped["MP_sum"]
splits_grouped["BLK_min"] = splits_grouped["BLK_sum"] / splits_grouped["MP_sum"]
splits_grouped["TOV_min"] = splits_grouped["TOV_sum"] / splits_grouped["MP_sum"]
splits_grouped.head()

,Link,Name,Split,Value,G_sum,MP_sum,Three_P_sum,STL_sum,BLK_sum,TOV_sum,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min
0,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Atlanta,17,354,8,10,10,15,20.823529,0.470588,0.588235,0.588235,0.882353,0.022599,0.028249,0.028249,0.042373
1,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Boston,25,561,8,14,19,19,22.440000,0.320000,0.560000,0.760000,0.760000,0.014260,0.024955,0.033868,0.033868
2,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Brooklyn,17,240,7,4,1,13,14.117647,0.411765,0.235294,0.058824,0.764706,0.029167,0.016667,0.004167,0.054167
3,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Charlotte,20,398,25,18,6,26,19.900000,1.250000,0.900000,0.300000,1.300000,0.062814,0.045226,0.015075,0.065327
4,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Chicago,25,444,4,6,20,30,17.760000,0.160000,0.240000,0.800000,1.200000,0.009009,0.013514,0.045045,0.067568


### We can filter what split to use with this line

In [65]:
splits_grouped.loc[splits_grouped["Split"] == "Total"]

,Link,Name,Split,Value,G_sum,MP_sum,Three_P_sum,STL_sum,BLK_sum,TOV_sum,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min
34,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Total,Total,405,8877,192,237,305,451,21.918519,0.474074,0.585185,0.753086,1.113580,0.021629,0.026698,0.034358,0.050805
69,https://www.basketball-reference.com/players/a/adamsst01.html,Steven Adams,Total,Total,218,5870,0,191,190,351,26.926606,0.000000,0.876147,0.871560,1.610092,0.000000,0.032538,0.032368,0.059796
103,https://www.basketball-reference.com/players/a/adebaba01.html,Bam Adebayo,Total,Total,419,14269,47,499,364,1008,34.054893,0.112172,1.190931,0.868735,2.405728,0.003294,0.034971,0.025510,0.070643
137,https://www.basketball-reference.com/players/a/agbajoc01.html,Ochai Agbaji,Total,Total,352,7344,348,173,162,274,20.863636,0.988636,0.491477,0.460227,0.778409,0.047386,0.023557,0.022059,0.037309
171,https://www.basketball-reference.com/players/a/aldamsa01.html,Santi Aldama,Total,Total,369,8578,512,225,268,343,23.246612,1.387534,0.609756,0.726287,0.929539,0.059688,0.026230,0.031243,0.039986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22382,https://www.basketball-reference.com/players/y/youngth01.html,Thaddeus Young,Total,Total,259,3752,32,231,43,181,14.486486,0.123552,0.891892,0.166023,0.698842,0.008529,0.061567,0.011461,0.048241
22416,https://www.basketball-reference.com/players/y/youngtr01.html,Trae Young,Total,Total,384,13560,1066,448,58,1608,35.312500,2.776042,1.166667,0.151042,4.187500,0.078614,0.033038,0.004277,0.118584
22451,https://www.basketball-reference.com/players/y/yurtsom01.html,Omer Yurtseven,Total,Total,218,2507,22,45,78,160,11.500000,0.100917,0.206422,0.357798,0.733945,0.008775,0.017950,0.031113,0.063821
22485,https://www.basketball-reference.com/players/z/zelleco01.html,Cody Zeller,Total,Total,186,1749,3,41,29,95,9.403226,0.016129,0.220430,0.155914,0.510753,0.001715,0.023442,0.016581,0.054317


### We can filter player and specific split values

We can see relevant info for a selected player.


In [66]:
player = splits_grouped.loc[splits_grouped["Name"] == "Precious Achiuwa"]
values = player.loc[player["Value"].isin(["Win", "Road", "Golden State"])]
values

,Link,Name,Split,Value,G_sum,MP_sum,Three_P_sum,STL_sum,BLK_sum,TOV_sum,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min
9,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Opponent,Golden State,10,253,2,3,6,15,25.300000,0.200000,0.300000,0.600000,1.500000,0.007905,0.011858,0.023715,0.059289
31,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Place,Road,187,3936,92,104,111,199,21.048128,0.491979,0.556150,0.593583,1.064171,0.023374,0.026423,0.028201,0.050559
33,https://www.basketball-reference.com/players/a/achiupr01.html,Precious Achiuwa,Result,Win,226,4743,111,152,177,252,20.986726,0.491150,0.672566,0.783186,1.115044,0.023403,0.032047,0.037318,0.053131


In [67]:
stats = values[["MP_game", "3P_game", "STL_game", "BLK_game", "TOV_game", "3P_min", "STL_min", "BLK_min", "TOV_min"]].sum()/3
stats

MP_game     22.444951
3P_game      0.394376
STL_game     0.509572
BLK_game     0.658923
TOV_game     1.226405
3P_min       0.018227
STL_min      0.023443
BLK_min      0.029745
TOV_min      0.054326
dtype: float64

# Now we have to connect these predicted values to Odds DataFrame

#### First we create column Result from Visitor and Home PTS 

In [68]:
odds["Result"] = np.where(odds['Visitor PTS'] > odds['Home PTS'], 'Visitor', "Home")
odds.head()

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home


#### Teams in Splits use different names than in Odds so we have to map them using dictionary

In [69]:
team_name_mapping = {
    "Detroit Pistons": "Detroit",
    "Milwaukee Bucks": "Milwaukee",
    "San Antonio Spurs": "San Antonio",
    "Washington Wizards": "Washington",
    "Sacramento Kings": "Sacramento",
    "Denver Nuggets": "Denver",
    "Portland Trail Blazers": "Portland",
    "Charlotte Hornets": "Charlotte",
    "Houston Rockets": "Houston",
    "Los Angeles Clippers": "LA Clippers",
    "New Orleans Pelicans": "New Orleans",
    "Brooklyn Nets": "Brooklyn",
    "Cleveland Cavaliers": "Cleveland",
    "Los Angeles Lakers": "LA Lakers",
    "Philadelphia 76ers": "Philadelphia",
    "Phoenix Suns": "Phoenix",
    "Chicago Bulls": "Chicago",
    "Atlanta Hawks": "Atlanta",
    "Utah Jazz": "Utah",
    "Memphis Grizzlies": "Memphis",
    "Minnesota Timberwolves": "Minnesota",
    "Toronto Raptors": "Toronto",
    "Dallas Mavericks": "Dallas",
    "Boston Celtics": "Boston",
    "New York Knicks": "New York",
    "Indiana Pacers": "Indiana",
    "Orlando Magic": "Orlando",
    "Golden State Warriors": "Golden State",
    "Miami Heat": "Miami",
    "Oklahoma City Thunder": "Oklahoma City"
}

In [70]:
# Add new columns to fill
new_columns = ["MP_game", "3P_game", "STL_game", "BLK_game", "TOV_game", "3P_min", "STL_min", "BLK_min", "TOV_min"]
for col in new_columns:
    odds[col] = None
# Calculate predicted values
for ind in tqdm(odds.index):
    player = splits_grouped.loc[splits_grouped["Link"] == odds["Player Link"][ind]]
    # Result
    if odds["Result"][ind] == odds["Side"][ind]:
        result = "Win"
    else:
        result = "Loss"
    # Place
    if odds["Side"][ind] == "Home":
        place = "Home"
    else:
        place = "Road"
    # Opponent
    if odds["Side"][ind] == "Visitor":
        opponent = team_name_mapping[odds["Home"][ind]]
        if opponent == "Oklahoma City":
            if player.loc[player["Value"] == opponent].empty:
                opponent = "OKC, SEA"
    else:
        opponent = team_name_mapping[odds["Visitor"][ind]]
        if opponent == "Oklahoma City":
            if player.loc[player["Value"] == opponent].empty:
                opponent = "OKC, SEA"
                
    values = player.loc[player["Value"].isin([result, place, opponent])]
    stats = values[["MP_game", "3P_game", "STL_game", "BLK_game", "TOV_game", "3P_min", "STL_min", "BLK_min", "TOV_min"]].sum()/3
    odds.loc[ind, ["MP_game", "3P_game", "STL_game", "BLK_game", "TOV_game", "3P_min", "STL_min", "BLK_min", "TOV_min"]] = stats.loc[["MP_game", "3P_game", "STL_game", "BLK_game", "TOV_game", "3P_min", "STL_min", "BLK_min", "TOV_min"]]

100%|███████████████████████████████████████████████████████████████████████████| 57586/57586 [05:58<00:00, 160.56it/s]


In [75]:
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home,3.103896,0.000000,0.077922,0.000000,0.060606,0.000000,0.016607,0.000000,0.013605
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home,13.042382,0.768114,0.284586,0.184280,0.336870,0.061167,0.021821,0.014275,0.024340
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home,34.891221,0.329336,1.174912,2.204486,1.881568,0.009397,0.033712,0.063210,0.053773
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home,29.547192,1.654676,0.596476,0.242672,1.619453,0.055640,0.020181,0.008190,0.055086
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home,4.600000,0.133333,0.400000,0.000000,0.200000,0.019360,0.058081,0.000000,0.028620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html,Visitor,Home,15.513019,0.027331,0.425419,0.559974,0.680344,0.001608,0.027338,0.039910,0.043891
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Olivier-Maxence Prosper,18,https://www.basketball-reference.com/players/p/prospol01.html,Visitor,Home,8.599206,0.251984,0.500992,0.036706,0.220238,0.023839,0.098521,0.003388,0.020330
57583,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball

In [76]:
odds.to_csv("odds.csv", index = False)

In [96]:
odds = pd.read_csv("odds.csv")

In [97]:
odds["TOV_min"].mean()

0.04823048150795662

# Now we project team strenght into the predicted values

In [11]:
teams = pd.read_csv("teams.csv")
teams[["Team", "3P", "STL", "BLK", "TOV"]]

,Team,3P,STL,BLK,TOV
0,Minnesota Timberwolves,937,616,371,1166
1,New York Knicks,1028,552,431,1082
2,Miami Heat,1064,543,388,1126
3,Orlando Magic,945,666,379,1234
4,Boston Celtics,1059,510,304,986
5,Denver Nuggets,919,578,394,1014
6,Cleveland Cavaliers,1029,632,411,1115
7,New Orleans Pelicans,1109,536,424,1168
8,Philadelphia 76ers,979,506,503,1201
9,Los Angeles Clippers,1054,597,384,1069


In [12]:
teams = teams[["Team", "3P", "STL", "BLK", "TOV"]]
teams_compare = pd.read_csv("teams.csv")[["Team", "3P", "STL", "BLK", "TOV"]]

In [13]:
teams_compare[["3P", "STL", "BLK", "TOV"]] = teams[["3P", "STL", "BLK", "TOV"]]/teams[["3P", "STL", "BLK", "TOV"]].mean()

#### We have table with strenghts of teams in different stats

In [14]:
teams_compare

,Team,3P,STL,BLK,TOV
0,Minnesota Timberwolves,0.890149,1.005058,0.879911,1.045177
1,New York Knicks,0.976598,0.900636,1.022215,0.969882
2,Miami Heat,1.010798,0.885952,0.920231,1.009322
3,Orlando Magic,0.897749,1.086637,0.898885,1.106131
4,Boston Celtics,1.006048,0.832110,0.721006,0.883829
5,Denver Nuggets,0.873049,0.943058,0.934461,0.908928
6,Cleveland Cavaliers,0.977548,1.031163,0.974781,0.999462
7,New Orleans Pelicans,1.053548,0.874531,1.005613,1.046970
8,Philadelphia 76ers,0.930048,0.825583,1.192980,1.076551
9,Los Angeles Clippers,1.001298,0.974058,0.910744,0.958229


#### Now we multiply these numbers with our predicted values

In [102]:
for ind in tqdm(odds.index):
    if odds["Side"][ind] == "Visitor":
        opponent = odds["Home"][ind]
    else:
        opponent = odds["Visitor"][ind]
    values = teams_compare.loc[teams_compare["Team"] == opponent]
    #stats = values[["MP_game", "3P_game", "STL_game", "BLK_game", "TOV_game", "3P_min", "STL_min", "BLK_min", "TOV_min"]].sum()/3
    odds.loc[ind, ["3P_game", "3P_min"]] = odds.loc[ind, ["3P_game", "3P_min"]]*values["3P"].iloc[0]
    odds.loc[ind, ["STL_game", "STL_min"]] = odds.loc[ind, ["STL_game", "STL_min"]]*values["STL"].iloc[0]
    odds.loc[ind, ["BLK_game", "BLK_min"]] = odds.loc[ind, ["BLK_game", "BLK_min"]]*values["BLK"].iloc[0]
    odds.loc[ind, ["TOV_game", "TOV_min"]] = odds.loc[ind, ["TOV_game", "TOV_min"]]*values["TOV"].iloc[0]

100%|███████████████████████████████████████████████████████████████████████████| 57586/57586 [07:12<00:00, 133.15it/s]


In [103]:
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home,3.103896,0.000000,0.073485,0.000000,0.055087,0.000000,0.015661,0.000000,0.012366
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home,13.042382,0.670601,0.268381,0.172202,0.306191,0.053402,0.020579,0.013339,0.022123
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home,34.891221,0.287526,1.108010,2.060006,1.710210,0.008204,0.031792,0.059067,0.048876
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home,29.547192,1.444613,0.562511,0.226768,1.471966,0.048576,0.019032,0.007653,0.050069
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home,4.600000,0.116406,0.377223,0.000000,0.181786,0.016902,0.054774,0.000000,0.026013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html,Visitor,Home,15.513019,0.027496,0.353995,0.403745,0.601308,0.001617,0.022748,0.028776,0.038792
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Olivier-Maxence Prosper,18,https://www.basketball-reference.com/players/p/prospol01.html,Visitor,Home,8.599206,0.253508,0.416880,0.026465,0.194653,0.023983,0.081980,0.002443,0.017968
57583,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball

In [104]:
odds.to_csv("odds_new.csv", index = False)

## Create over/under values

In [105]:
odds = pd.read_csv("odds_new.csv")

In [106]:
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home,3.103896,0.000000,0.073485,0.000000,0.055087,0.000000,0.015661,0.000000,0.012366
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home,13.042382,0.670601,0.268381,0.172202,0.306191,0.053402,0.020579,0.013339,0.022123
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home,34.891221,0.287526,1.108010,2.060006,1.710210,0.008204,0.031792,0.059067,0.048876
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home,29.547192,1.444613,0.562511,0.226768,1.471966,0.048576,0.019032,0.007653,0.050069
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home,4.600000,0.116406,0.377223,0.000000,0.181786,0.016902,0.054774,0.000000,0.026013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html,Visitor,Home,15.513019,0.027496,0.353995,0.403745,0.601308,0.001617,0.022748,0.028776,0.038792
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Olivier-Maxence Prosper,18,https://www.basketball-reference.com/players/p/prospol01.html,Visitor,Home,8.599206,0.253508,0.416880,0.026465,0.194653,0.023983,0.081980,0.002443,0.017968
57583,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball

In [107]:
# Define the rounding function
def round_to_nearest_half(number):
    rounded = round(number * 2) / 2
    # If rounded result is a whole number, adjust to the nearest 0.5
    if rounded % 1 == 0:
        if number >= rounded:
            return rounded + 0.5
        else:
            return rounded - 0.5
    return rounded

In [108]:
# Apply the custom rounding function to create a new column
odds["3P over/under"] = (odds["MP_game"]*odds["3P_min"]).apply(round_to_nearest_half)
odds["STL over/under"] = (odds["MP_game"]*odds["STL_min"]).apply(round_to_nearest_half)
odds["BLK over/under"] = (odds["MP_game"]*odds["BLK_min"]).apply(round_to_nearest_half)
odds["TOV over/under"] = (odds["MP_game"]*odds["TOV_min"]).apply(round_to_nearest_half)

In [109]:
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min,3P over/under,STL over/under,BLK over/under,TOV over/under
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home,3.103896,0.000000,0.073485,0.000000,0.055087,0.000000,0.015661,0.000000,0.012366,0.5,0.5,0.5,0.5
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home,13.042382,0.670601,0.268381,0.172202,0.306191,0.053402,0.020579,0.013339,0.022123,0.5,0.5,0.5,0.5
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home,34.891221,0.287526,1.108010,2.060006,1.710210,0.008204,0.031792,0.059067,0.048876,0.5,1.5,2.5,1.5
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home,29.547192,1.444613,0.562511,0.226768,1.471966,0.048576,0.019032,0.007653,0.050069,1.5,0.5,0.5,1.5
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home,4.600000,0.116406,0.377223,0.000000,0.181786,0.016902,0.054774,0.000000,0.026013,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html,Visitor,Home,15.513019,0.027496,0.353995,0.403745,0.601308,0.001617,0.022748,0.028776,0.038792,0.5,0.5,0.5,0.5
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Olivier-Maxence Prosper,18,https://www.basketball-reference.com/players/p/prospol01.html,Visitor,Home,8.599206,0.253508,0.416880,0.026465,0.194653,0.023983,0.081980,0.002443,0.017968,0.5,0.5,0.5,0.5
57583,"Mon, Ju

# Add odds for over/under values

In [110]:
gamelogs = pd.read_csv("gamelogs_trans.csv")
gamelogs

,Name,Link,Game,Date,Match Link,Age,Team,Location,Opponent,Result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Diff
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,1,2023-10-25,https://www.basketball-reference.com/boxscores/202310250TOR.html,24-036,TOR,Road,MIN,W,0,24:17,4,9,0.444,0,1,0.000,0,0,0.000,3,5,8,0,0,0,2,1,8,4.5,-5.0,3
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2,2023-10-27,https://www.basketball-reference.com/boxscores/202310270CHI.html,24-038,TOR,Home,CHI,L,0,22:38,2,5,0.400,1,3,0.333,3,3,1.000,2,7,9,1,0,0,3,6,8,4.1,-22.0,-1
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,3,2023-10-28,https://www.basketball-reference.com/boxscores/202310280TOR.html,24-039,TOR,Road,PHI,L,0,18:15,3,10,0.300,0,1,0.000,0,0,0.000,3,4,7,3,0,1,1,1,6,4.9,5.0,-7
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,4,2023-11-11,https://www.basketball-reference.com/boxscores/202311110BOS.html,24-053,TOR,Home,BOS,L,0,18:31,1,5,0.200,0,2,0.000,0,0,0.000,0,5,5,5,0,0,0,1,2,3.5,-12.0,-23
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,5,2023-11-13,https://www.basketball-reference.com/boxscores/202311130TOR.html,24-055,TOR,Road,WAS,W,0,19:16,5,8,0.625,0,1,0.000,0,0,0.000,1,3,4,1,1,0,2,1,10,7.3,8.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86528,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,1,2023-04-16,https://www.basketball-reference.com/boxscores/202304160PHO.html,26-029,LAC,Home,PHO,W,1,29:43,5,8,0.625,0,0,0.000,2,4,0.500,4,11,15,0,0,0,3,3,12,9.5,-11.0,5
86529,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2,2023-04-18,https://www.basketball-reference.com/boxscores/202304180PHO.html,26-031,LAC,Home,PHO,L,1,29:29,2,7,0.286,0,0,0.000,4,6,0.667,4,2,6,0,2,0,3,1,8,5.1,-14.0,-14
86530,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,3,2023-04-20,https://www.basketball-reference.com/boxscores/202304200LAC.html,26-033,LAC,Road,PHO,L,1,22:20,3,4,0.750,0,0,0.000,0,0,0.000,1,7,8,0,0,0,2,1,6,4.8,-5.0,-5
86531,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,4,2023-04-22,https://www.basketball-reference.com/boxscores/202304220LAC.html,26-035,LAC,Road,PHO,L,1,24:25,2,5,0.400,0,0,0.000,0,0,0.000,3,6,9,1,1,0,1,1,4,5.5,-8.0,-12


### Change MP to contain decimal numbers

In [111]:
def time_to_decimal(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes + seconds / 60

In [112]:
# Replace seconds with decimal part
gamelogs['MP'] = gamelogs['MP'].apply(time_to_decimal)

In [113]:
gamelogs

,Name,Link,Game,Date,Match Link,Age,Team,Location,Opponent,Result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Diff
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,1,2023-10-25,https://www.basketball-reference.com/boxscores/202310250TOR.html,24-036,TOR,Road,MIN,W,0,24.283333,4,9,0.444,0,1,0.000,0,0,0.000,3,5,8,0,0,0,2,1,8,4.5,-5.0,3
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2,2023-10-27,https://www.basketball-reference.com/boxscores/202310270CHI.html,24-038,TOR,Home,CHI,L,0,22.633333,2,5,0.400,1,3,0.333,3,3,1.000,2,7,9,1,0,0,3,6,8,4.1,-22.0,-1
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,3,2023-10-28,https://www.basketball-reference.com/boxscores/202310280TOR.html,24-039,TOR,Road,PHI,L,0,18.250000,3,10,0.300,0,1,0.000,0,0,0.000,3,4,7,3,0,1,1,1,6,4.9,5.0,-7
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,4,2023-11-11,https://www.basketball-reference.com/boxscores/202311110BOS.html,24-053,TOR,Home,BOS,L,0,18.516667,1,5,0.200,0,2,0.000,0,0,0.000,0,5,5,5,0,0,0,1,2,3.5,-12.0,-23
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,5,2023-11-13,https://www.basketball-reference.com/boxscores/202311130TOR.html,24-055,TOR,Road,WAS,W,0,19.266667,5,8,0.625,0,1,0.000,0,0,0.000,1,3,4,1,1,0,2,1,10,7.3,8.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86528,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,1,2023-04-16,https://www.basketball-reference.com/boxscores/202304160PHO.html,26-029,LAC,Home,PHO,W,1,29.716667,5,8,0.625,0,0,0.000,2,4,0.500,4,11,15,0,0,0,3,3,12,9.5,-11.0,5
86529,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2,2023-04-18,https://www.basketball-reference.com/boxscores/202304180PHO.html,26-031,LAC,Home,PHO,L,1,29.483333,2,7,0.286,0,0,0.000,4,6,0.667,4,2,6,0,2,0,3,1,8,5.1,-14.0,-14
86530,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,3,2023-04-20,https://www.basketball-reference.com/boxscores/202304200LAC.html,26-033,LAC,Road,PHO,L,1,22.333333,3,4,0.750,0,0,0.000,0,0,0.000,1,7,8,0,0,0,2,1,6,4.8,-5.0,-5
86531,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,4,2023-04-22,https://www.basketball-reference.com/boxscores/202304220LAC.html,26-035,LAC,Road,PHO,L,1,24.416667,2,5,0.400,0,0,0.000,0,0,0.000,3,6,9,1,1,0,1,1,4,5.5,-8.0,-12


### Multiply rows to give weight 3 to 2024, weight 2 to 2023 and weight 1 to all other seasons

In [114]:
gamelogs['Date'] = pd.to_datetime(gamelogs['Date'])

#### 2024

In [115]:
# Filter rows for 2024 season
filtered_df = gamelogs[(gamelogs['Date'] >= "2023-09-01") & (gamelogs['Date'] <= "2024-07-01")]
filtered_df

,Name,Link,Game,Date,Match Link,Age,Team,Location,Opponent,Result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Diff
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,1,2023-10-25,https://www.basketball-reference.com/boxscores/202310250TOR.html,24-036,TOR,Road,MIN,W,0,24.283333,4,9,0.444,0,1,0.000,0,0,0.000,3,5,8,0,0,0,2,1,8,4.5,-5.0,3
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2,2023-10-27,https://www.basketball-reference.com/boxscores/202310270CHI.html,24-038,TOR,Home,CHI,L,0,22.633333,2,5,0.400,1,3,0.333,3,3,1.000,2,7,9,1,0,0,3,6,8,4.1,-22.0,-1
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,3,2023-10-28,https://www.basketball-reference.com/boxscores/202310280TOR.html,24-039,TOR,Road,PHI,L,0,18.250000,3,10,0.300,0,1,0.000,0,0,0.000,3,4,7,3,0,1,1,1,6,4.9,5.0,-7
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,4,2023-11-11,https://www.basketball-reference.com/boxscores/202311110BOS.html,24-053,TOR,Home,BOS,L,0,18.516667,1,5,0.200,0,2,0.000,0,0,0.000,0,5,5,5,0,0,0,1,2,3.5,-12.0,-23
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,5,2023-11-13,https://www.basketball-reference.com/boxscores/202311130TOR.html,24-055,TOR,Road,WAS,W,0,19.266667,5,8,0.625,0,1,0.000,0,0,0.000,1,3,4,1,1,0,2,1,10,7.3,8.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86523,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2,2024-04-23,https://www.basketball-reference.com/boxscores/202404230LAC.html,27-036,LAC,Road,DAL,L,1,33.066667,5,12,0.417,0,0,0.000,3,5,0.600,5,7,12,0,1,0,2,2,13,9.6,4.0,-3
86524,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,3,2024-04-26,https://www.basketball-reference.com/boxscores/202404260DAL.html,27-039,LAC,Home,DAL,L,1,32.283333,8,11,0.727,0,0,0.000,3,3,1.000,3,5,8,1,1,1,2,4,19,16.9,-5.0,-11
86525,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,4,2024-04-28,https://www.basketball-reference.com/boxscores/202404280DAL.html,27-041,LAC,Home,DAL,W,1,30.266667,6,11,0.545,0,0,0.000,1,2,0.500,1,3,4,1,1,1,1,4,13,8.7,-1.0,5
86526,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,5,2024-05-01,https://www.basketball-reference.com/boxscores/202405010LAC.html,27-044,LAC,Road,DAL,L,1,25.683333,7,8,0.875,0,0,0.000,1,1,1.000,1,5,6,2,1,0,1,4,15,14.2,-18.0,-30


In [116]:
# Concatenate 2024 season twice to gamelogs
gamelogs = pd.concat([gamelogs, filtered_df], ignore_index=True)
gamelogs = pd.concat([gamelogs, filtered_df], ignore_index=True)

In [117]:
gamelogs

,Name,Link,Game,Date,Match Link,Age,Team,Location,Opponent,Result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Diff
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,1,2023-10-25,https://www.basketball-reference.com/boxscores/202310250TOR.html,24-036,TOR,Road,MIN,W,0,24.283333,4,9,0.444,0,1,0.000,0,0,0.000,3,5,8,0,0,0,2,1,8,4.5,-5.0,3
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2,2023-10-27,https://www.basketball-reference.com/boxscores/202310270CHI.html,24-038,TOR,Home,CHI,L,0,22.633333,2,5,0.400,1,3,0.333,3,3,1.000,2,7,9,1,0,0,3,6,8,4.1,-22.0,-1
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,3,2023-10-28,https://www.basketball-reference.com/boxscores/202310280TOR.html,24-039,TOR,Road,PHI,L,0,18.250000,3,10,0.300,0,1,0.000,0,0,0.000,3,4,7,3,0,1,1,1,6,4.9,5.0,-7
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,4,2023-11-11,https://www.basketball-reference.com/boxscores/202311110BOS.html,24-053,TOR,Home,BOS,L,0,18.516667,1,5,0.200,0,2,0.000,0,0,0.000,0,5,5,5,0,0,0,1,2,3.5,-12.0,-23
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,5,2023-11-13,https://www.basketball-reference.com/boxscores/202311130TOR.html,24-055,TOR,Road,WAS,W,0,19.266667,5,8,0.625,0,1,0.000,0,0,0.000,1,3,4,1,1,0,2,1,10,7.3,8.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142698,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2,2024-04-23,https://www.basketball-reference.com/boxscores/202404230LAC.html,27-036,LAC,Road,DAL,L,1,33.066667,5,12,0.417,0,0,0.000,3,5,0.600,5,7,12,0,1,0,2,2,13,9.6,4.0,-3
142699,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,3,2024-04-26,https://www.basketball-reference.com/boxscores/202404260DAL.html,27-039,LAC,Home,DAL,L,1,32.283333,8,11,0.727,0,0,0.000,3,3,1.000,3,5,8,1,1,1,2,4,19,16.9,-5.0,-11
142700,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,4,2024-04-28,https://www.basketball-reference.com/boxscores/202404280DAL.html,27-041,LAC,Home,DAL,W,1,30.266667,6,11,0.545,0,0,0.000,1,2,0.500,1,3,4,1,1,1,1,4,13,8.7,-1.0,5
142701,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,5,2024-05-01,https://www.basketball-reference.com/boxscores/202405010LAC.html,27-044,LAC,Road,DAL,L,1,25.683333,7,8,0.875,0,0,0.000,1,1,1.000,1,5,6,2,1,0,1,4,15,14.2,-18.0,-30


#### 2023

In [118]:
# Filter rows for 2023 season
filtered_df = gamelogs[(gamelogs['Date'] >= "2022-09-01") & (gamelogs['Date'] <= "2023-07-01")]
filtered_df

,Name,Link,Game,Date,Match Link,Age,Team,Location,Opponent,Result,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Diff
74,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,1,2022-10-19,https://www.basketball-reference.com/boxscores/202210190TOR.html,23-030,TOR,Road,CLE,W,0,17.933333,4,11,0.364,1,4,0.250,1,2,0.500,1,4,5,0,0,0,0,1,10,5.0,-4.0,3
75,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2,2022-10-21,https://www.basketball-reference.com/boxscores/202210210BRK.html,23-032,TOR,Home,BRK,L,0,17.483333,1,6,0.167,0,1,0.000,2,3,0.667,0,6,6,0,0,0,3,2,4,-2.2,-8.0,-4
76,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,3,2022-10-22,https://www.basketball-reference.com/boxscores/202210220MIA.html,23-033,TOR,Home,MIA,L,0,33.533333,5,9,0.556,2,5,0.400,6,7,0.857,3,8,11,1,0,0,1,1,18,17.1,16.0,-3
77,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,4,2022-10-24,https://www.basketball-reference.com/boxscores/202210240MIA.html,23-035,TOR,Home,MIA,W,0,33.716667,5,12,0.417,0,1,0.000,0,1,0.000,4,18,22,2,0,1,0,2,10,12.7,10.0,8
78,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,5,2022-10-26,https://www.basketball-reference.com/boxscores/202210260TOR.html,23-037,TOR,Road,PHI,W,0,21.300000,3,7,0.429,1,3,0.333,0,0,0.000,3,3,6,4,0,0,2,3,7,5.9,14.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86528,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,1,2023-04-16,https://www.basketball-reference.com/boxscores/202304160PHO.html,26-029,LAC,Home,PHO,W,1,29.716667,5,8,0.625,0,0,0.000,2,4,0.500,4,11,15,0,0,0,3,3,12,9.5,-11.0,5
86529,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2,2023-04-18,https://www.basketball-reference.com/boxscores/202304180PHO.html,26-031,LAC,Home,PHO,L,1,29.483333,2,7,0.286,0,0,0.000,4,6,0.667,4,2,6,0,2,0,3,1,8,5.1,-14.0,-14
86530,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,3,2023-04-20,https://www.basketball-reference.com/boxscores/202304200LAC.html,26-033,LAC,Road,PHO,L,1,22.333333,3,4,0.750,0,0,0.000,0,0,0.000,1,7,8,0,0,0,2,1,6,4.8,-5.0,-5
86531,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,4,2023-04-22,https://www.basketball-reference.com/boxscores/202304220LAC.html,26-035,LAC,Road,PHO,L,1,24.416667,2,5,0.400,0,0,0.000,0,0,0.000,3,6,9,1,1,0,1,1,4,5.5,-8.0,-12


In [119]:
# Concatenate 2023 season once to gamelogs
gamelogs = pd.concat([gamelogs, filtered_df], ignore_index=True)

In [120]:
gamelogs = gamelogs[["Name", "Link","Date", "Team", "Location", "Opponent", "Result", "MP", "3P", "STL", "BLK", "TOV"]]
gamelogs

,Name,Link,Date,Team,Location,Opponent,Result,MP,3P,STL,BLK,TOV
0,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2023-10-25,TOR,Road,MIN,W,24.283333,0,0,0,2
1,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2023-10-27,TOR,Home,CHI,L,22.633333,1,0,0,3
2,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2023-10-28,TOR,Road,PHI,L,18.250000,0,0,1,1
3,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2023-11-11,TOR,Home,BOS,L,18.516667,0,0,0,0
4,Precious Achiuwa,https://www.basketball-reference.com/players/a/achiupr01.html,2023-11-13,TOR,Road,WAS,W,19.266667,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
169350,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2023-04-16,LAC,Home,PHO,W,29.716667,0,0,0,3
169351,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2023-04-18,LAC,Home,PHO,L,29.483333,0,2,0,3
169352,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2023-04-20,LAC,Road,PHO,L,22.333333,0,0,0,2
169353,Ivica Zubac,https://www.basketball-reference.com/players/z/zubaciv01.html,2023-04-22,LAC,Road,PHO,L,24.416667,0,1,0,1


# Calculate probability for every row in odds

### Define function to get probabilities

In [121]:
def get_proba(player, stat):
    # Locate result gamelogs
    if odds["Result"][ind] == odds["Side"][ind]:
        result = player.loc[player["Result"] == "W"]
        over = result[result[stat] > odds[stat+" over/under"][ind] ][stat].count()
        count = result[stat].count()
        if count > 0:
            result_proba = over/count 
        else:
            result_proba = np.nan
    else:
        result = player.loc[player["Result"] == "L"]
        over = result[result[stat] > odds[stat+" over/under"][ind]][stat].count()
        count = result[stat].count()
        if count > 0:
            result_proba = over/count 
        else:
            result_proba = np.nan
    
    # Locate Location gamelogs
    if odds["Side"][ind] == "Home":
        location = player.loc[player["Location"] == "Home"]
        over = location[location[stat] > odds[stat+" over/under"][ind]][stat].count()
        count = location[stat].count()
        if count > 0:
            location_proba = over/count 
        else:
            location_proba = np.nan
    else:
        location = player.loc[player["Location"] == "Road"]
        over = location[location[stat] > odds[stat+" over/under"][ind]][stat].count()
        count = location[stat].count()
        if count > 0:
            location_proba = over/count 
        else:
            location_proba = np.nan

    # Locate Opponent gamelogs
    if odds["Side"][ind] == "Visitor":
        opp = odds["Home Link"][ind][-13:-10]
        opponent = player.loc[player["Opponent"] == opp]
        over = opponent[opponent[stat] > odds[stat+" over/under"][ind]][stat].count()
        count = opponent[stat].count()
        if count > 0:
            opponent_proba = over/count 
        else:
            opponent_proba = np.nan
    else:
        opp = odds["Visitor Link"][ind][-13:-10]  
        opponent = player.loc[player["Opponent"] == opp]
        over = opponent[opponent[stat] > odds[stat+" over/under"][ind]][stat].count()
        count = opponent[stat].count()
        if count > 0:
            opponent_proba = over/count 
        else:
            opponent_proba = np.nan
    # Compute the probabilities, ignoring NaN values
    if np.isnan(result_proba) and np.isnan(location_proba) and np.isnan(opponent_proba):
        proba = np.nan
    else:
        proba = np.nanmean([result_proba, location_proba, opponent_proba])
    return proba

In [122]:
new_columns = ["3P over proba", "STL over proba", "BLK over proba", "TOV over proba"]
for col in new_columns:
    odds[col] = None
# Calculate probabilities
for ind in tqdm(odds.index):
    # Locate player gamelogs
    player = gamelogs.loc[gamelogs["Link"] == odds["Player Link"][ind]]
    # Select only games that are within 5 mins from expected playtime
    player_mp = player[(player['MP'] >= odds["MP_game"][ind] - 5) & (player['MP'] <= odds["MP_game"][ind] + 5)]
    proba_3p = get_proba(player_mp, "3P")
    proba_stl = get_proba(player_mp, "STL")
    proba_blk = get_proba(player_mp, "BLK")
    proba_tov = get_proba(player_mp, "TOV")
    odds.loc[ind, ["3P over proba", "STL over proba", "BLK over proba", "TOV over proba"]] = [proba_3p, proba_stl, proba_blk, proba_tov]

100%|████████████████████████████████████████████████████████████████████████████| 57586/57586 [25:14<00:00, 38.03it/s]


In [123]:
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min,3P over/under,STL over/under,BLK over/under,TOV over/under,3P over proba,STL over proba,BLK over proba,TOV over proba
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home,3.103896,0.000000,0.073485,0.000000,0.055087,0.000000,0.015661,0.000000,0.012366,0.5,0.5,0.5,0.5,0.0,0.183333,0.0,0.1
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home,13.042382,0.670601,0.268381,0.172202,0.306191,0.053402,0.020579,0.013339,0.022123,0.5,0.5,0.5,0.5,0.699603,0.383862,0.221825,0.241402
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home,34.891221,0.287526,1.108010,2.060006,1.710210,0.008204,0.031792,0.059067,0.048876,0.5,1.5,2.5,1.5,0.235988,0.333491,0.383621,0.617701
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home,29.547192,1.444613,0.562511,0.226768,1.471966,0.048576,0.019032,0.007653,0.050069,1.5,0.5,0.5,1.5,0.533477,0.489625,0.194109,0.42347
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home,4.600000,0.116406,0.377223,0.000000,0.181786,0.016902,0.054774,0.000000,0.026013,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www.basketball-reference.com/players/p/poweldw01.html,Visitor,Home,15.513019,0.027496,0.353995,0.403745,0.601308,0.001617,0.022748,0.028776,0.038792,0.5,0.5,0.5,0.5,0.029448,0.381246,0.322992,0.527364
57582,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.ba

In [124]:
odds.to_csv("odds_proba.csv", index = False)

In [48]:
odds = pd.read_csv("odds_proba.csv")

In [49]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57586 entries, 0 to 57585
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            57586 non-null  object 
 1   Visitor         57586 non-null  object 
 2   Visitor PTS     57586 non-null  int64  
 3   Home            57586 non-null  object 
 4   Home PTS        57586 non-null  int64  
 5   Visitor Link    57586 non-null  object 
 6   Home Link       57586 non-null  object 
 7   Match Link      57586 non-null  object 
 8   Overtimes       2647 non-null   object 
 9   Team            57586 non-null  object 
 10  Team Link       57586 non-null  object 
 11  Player          57586 non-null  object 
 12  Number          57586 non-null  object 
 13  Player Link     57586 non-null  object 
 14  Side            57586 non-null  object 
 15  Result          57586 non-null  object 
 16  MP_game         57586 non-null  float64
 17  3P_game         57586 non-null 

In [50]:
odds.describe()

,Visitor PTS,Home PTS,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min,3P over/under,STL over/under,BLK over/under,TOV over/under,3P over proba,STL over proba,BLK over proba,TOV over proba
count,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57586.000000,57180.000000,57180.000000,57180.000000,57180.000000
mean,112.540201,114.737836,18.888329,0.951911,0.585273,0.389394,0.997943,0.044535,0.029532,0.020072,0.048274,1.053954,0.648178,0.581947,1.031501,0.330191,0.350986,0.253542,0.392823
std,12.895419,12.912119,9.515651,0.853238,0.396049,0.418375,0.784905,0.031961,0.018315,0.020176,0.027949,0.790704,0.364258,0.330402,0.784757,0.225166,0.207996,0.198547,0.200396
min,70.000000,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000
25%,104.000000,106.000000,11.623674,0.240685,0.283492,0.118349,0.445365,0.019932,0.019041,0.007268,0.031359,0.500000,0.500000,0.500000,0.500000,0.137409,0.191420,0.088216,0.257725
50%,113.000000,114.000000,18.405614,0.756361,0.526099,0.271275,0.803620,0.042797,0.027002,0.014344,0.044762,0.500000,0.500000,0.500000,0.500000,0.342657,0.354167,0.224747,0.402137
75%,122.000000,123.000000,26.776599,1.452696,0.820609,0.522026,1.343706,0.065134,0.036698,0.025870,0.061242,1.500000,0.500000,0.500000,1.500000,0.497577,0.503952,0.389292,0.532723
max,157.000000,157.000000,39.055423,6.077451,3.176534,5.728875,5.272936,0.449038,0.406901,0.321843,0.488200,6.500000,3.500000,5.500000,6.500000,1.000000,1.000000,1.000000,1.000000


In [51]:
odds["TOV over proba"].value_counts().head(10)

TOV over proba
0.000000    3327
0.500000     341
1.000000     279
0.250000     278
0.166667     270
0.125000     246
0.333333     238
0.750000     161
0.416667     155
0.083333     135
Name: count, dtype: int64

In [52]:
odds.loc[odds["Player"] == "LeBron James"].head(50)

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min,3P over/under,STL over/under,BLK over/under,TOV over/under,3P over proba,STL over proba,BLK over proba,TOV over proba
10,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/players/j/jamesle01.html,Visitor,Home,35.522604,1.766136,1.217506,0.512920,2.898465,0.049592,0.034380,0.014324,0.081369,1.5,1.5,0.5,2.5,0.647028,0.414217,0.530873,0.681147
667,"Thu, Oct 26, 2023",Phoenix Suns,95,Los Angeles Lakers,100,https://www.basketball-reference.com/teams/PHO/2024.html,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/boxscores/202310260LAL.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/players/j/jamesle01.html,Home,Home,34.709231,2.320328,1.518626,0.536496,3.244422,0.066860,0.043952,0.015472,0.093588,2.5,1.5,0.5,3.5,0.461163,0.440844,0.670782,0.588735
1728,"Sun, Oct 29, 2023",Los Angeles Lakers,127,Sacramento Kings,132,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/SAC/2024.html,https://www.basketball-reference.com/boxscores/202310290SAC.html,OT,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/players/j/jamesle01.html,Visitor,Home,36.750778,2.154101,1.210765,0.543487,3.820954,0.058699,0.032955,0.014825,0.103797,2.5,1.5,0.5,3.5,0.370158,0.330512,0.491602,0.479717
2215,"Mon, Oct 30, 2023",Orlando Magic,103,Los Angeles Lakers,106,https://www.basketball-reference.com/teams/ORL/2024.html,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/boxscores/202310300LAL.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/players/j/jamesle01.html,Home,Home,34.642564,1.877042,1.567056,0.621499,4.078221,0.054063,0.045538,0.018020,0.118080,1.5,1.5,0.5,4.5,0.660211,0.421538,0.622711,0.337119
2906,"Wed, Nov 1, 2023",Los Angeles Clippers,125,Los Angeles Lakers,130,https://www.basketball-reference.com/teams/LAC/2024.html,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/boxscores/202311010LAL.html,OT,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/players/j/jamesle01.html,Home,Home,35.692564,2.861210,1.063783,0.690414,3.213506,0.079692,0.029839,0.019212,0.090059,2.5,1.5,0.5,3.5,0.401660,0.337054,0.615307,0.489149
3580,"Sat, Nov 4, 2023",Los Angeles Lakers,101,Orlando Magic,120,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/ORL/2024.html,https://www.basketball-reference.com/boxscores/202311040ORL.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/players/j/jamesle01.html,Visitor,Home,35.380937,1.819843,1.642836,0.669425,4.181785,0.051292,0.046928,0.019037,0.118801,1.5,1.5,0.5,4.5,0.710796,0.425811,0.561308,0.323852
4238,"Mon, Nov 6, 2023",Los Angeles Lakers,107,Miami Heat,108,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/MIA/2024.html,https://www.basketball-reference.com/boxscores/202311060MIA.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,LeBron James,23,https://www.basketball-reference.com/play

# Now we project team strenght in probabilities as we did with predicted values

In [53]:
for ind in tqdm(odds.index):
    if odds["Side"][ind] == "Visitor":
        opponent = odds["Home"][ind]
    else:
        opponent = odds["Visitor"][ind]
    values = teams_compare.loc[teams_compare["Team"] == opponent]

    odds.loc[ind, "3P over proba"] = odds.loc[ind, "3P over proba"]*values["3P"].iloc[0]
    odds.loc[ind, "STL over proba"] = odds.loc[ind, "STL over proba"]*values["STL"].iloc[0]
    odds.loc[ind, "BLK over proba"] = odds.loc[ind, "BLK over proba"]*values["BLK"].iloc[0]
    odds.loc[ind, "TOV over proba"] = odds.loc[ind, "TOV over proba"]*values["TOV"].iloc[0]

100%|███████████████████████████████████████████████████████████████████████████| 57586/57586 [01:16<00:00, 749.14it/s]


# Create over/under odds from probabilities

In [54]:
odds["3P over"] = 1/odds["3P over proba"]
odds["3P under"] = 1/(1-odds["3P over proba"])
odds["STL over"] = 1/odds["STL over proba"]
odds["STL under"] = 1/(1-odds["STL over proba"])
odds["BLK over"] = 1/odds["BLK over proba"]
odds["BLK under"] = 1/(1-odds["BLK over proba"])
odds["TOV over"] = 1/odds["TOV over proba"]
odds["TOV under"] = 1/(1-odds["TOV over proba"])
odds

,Date,Visitor,Visitor PTS,Home,Home PTS,Visitor Link,Home Link,Match Link,Overtimes,Team,Team Link,Player,Number,Player Link,Side,Result,MP_game,3P_game,STL_game,BLK_game,TOV_game,3P_min,STL_min,BLK_min,TOV_min,3P over/under,STL over/under,BLK over/under,TOV over/under,3P over proba,STL over proba,BLK over proba,TOV over proba,3P over,3P under,STL over,STL under,BLK over,BLK under,TOV over,TOV under
0,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Colin Castleton,14,https://www.basketball-reference.com/players/c/castlco01.html,Visitor,Home,3.103896,0.000000,0.073485,0.000000,0.055087,0.000000,0.015661,0.000000,0.012366,0.5,0.5,0.5,0.5,0.000000,0.172894,0.000000,0.090893,inf,1.000000,5.783894,1.209035,inf,1.000000,11.001972,1.099980
1,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Max Christie,10,https://www.basketball-reference.com/players/c/chrisma02.html,Visitor,Home,13.042382,0.670601,0.268381,0.172202,0.306191,0.053402,0.020579,0.013339,0.022123,0.5,0.5,0.5,0.5,0.610788,0.362004,0.207287,0.219417,1.637231,2.569291,2.762397,1.567409,4.824224,1.261491,4.557529,1.281094
2,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Anthony Davis,3,https://www.basketball-reference.com/players/d/davisan02.html,Visitor,Home,34.891221,0.287526,1.108010,2.060006,1.710210,0.008204,0.031792,0.059067,0.048876,0.5,1.5,2.5,1.5,0.206029,0.314501,0.358479,0.561446,4.853688,1.259492,3.179641,1.458791,2.789564,1.558795,1.781115,2.280221
3,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Spencer Dinwiddie,26,https://www.basketball-reference.com/players/d/dinwisp01.html,Visitor,Home,29.547192,1.444613,0.562511,0.226768,1.471966,0.048576,0.019032,0.007653,0.050069,1.5,0.5,0.5,1.5,0.465752,0.461745,0.181387,0.384904,2.147068,1.871788,2.165699,1.857854,5.513070,1.221579,2.598054,1.625761
4,"Tue, Oct 24, 2023",Los Angeles Lakers,107,Denver Nuggets,119,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/boxscores/202310240DEN.html,NaN,Los Angeles Lakers,https://www.basketball-reference.com/teams/LAL/2024.html,Alex Fudge,17,https://www.basketball-reference.com/players/f/fudgeal01.html,Visitor,Home,4.600000,0.116406,0.377223,0.000000,0.181786,0.016902,0.054774,0.000000,0.026013,0.5,0.5,0.5,0.5,0.000000,0.000000,0.000000,0.681696,inf,1.000000,inf,1.000000,inf,1.000000,1.466930,3.141650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57581,"Mon, Jun 17, 2024",Dallas Mavericks,88,Boston Celtics,106,https://www.basketball-reference.com/teams/DAL/2024.html,https://www.basketball-reference.com/teams/BOS/2024.html,https://www.basketball-reference.com/boxscores/202406170BOS.html,NaN,Dallas Mavericks,https://www.basketball-reference.com/teams/DAL/2024.html,Dwight Powell,7,https://www

### Replace missing values or sure odds with 0

In [55]:
odds[["3P over", "3P under", "STL over", "STL under", "BLK over", "BLK under", "TOV over", "TOV under"]] = odds[["3P over", "3P under", "STL over", "STL under", "BLK over", "BLK under", "TOV over", "TOV under"]].replace(1, 0)
odds[["3P over", "3P under", "STL over", "STL under", "BLK over", "BLK under", "TOV over", "TOV under"]] = odds[["3P over", "3P under", "STL over", "STL under", "BLK over", "BLK under", "TOV over", "TOV under"]].replace(np.inf, 0)
odds[["3P over", "3P under", "STL over", "STL under", "BLK over", "BLK under", "TOV over", "TOV under"]] = odds[["3P over", "3P under", "STL over", "STL under", "BLK over", "BLK under", "TOV over", "TOV under"]].replace(np.NaN, 0)

In [56]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57586 entries, 0 to 57585
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            57586 non-null  object 
 1   Visitor         57586 non-null  object 
 2   Visitor PTS     57586 non-null  int64  
 3   Home            57586 non-null  object 
 4   Home PTS        57586 non-null  int64  
 5   Visitor Link    57586 non-null  object 
 6   Home Link       57586 non-null  object 
 7   Match Link      57586 non-null  object 
 8   Overtimes       2647 non-null   object 
 9   Team            57586 non-null  object 
 10  Team Link       57586 non-null  object 
 11  Player          57586 non-null  object 
 12  Number          57586 non-null  object 
 13  Player Link     57586 non-null  object 
 14  Side            57586 non-null  object 
 15  Result          57586 non-null  object 
 16  MP_game         57586 non-null  float64
 17  3P_game         57586 non-null 

# Make dataframe "pretty"

In [57]:
odds = odds [["Date", "Visitor", "Home", "Result", "Side", "Player", "Number", "MP_game",
             "3P_game", "3P_min", "3P over/under", "3P over proba", "3P over", "3P under",
             "STL_game", "STL_min", "STL over/under", "STL over proba", "STL over", "STL under",
             "BLK_game", "BLK_min", "BLK over/under", "BLK over proba", "BLK over", "BLK under",
             "TOV_game", "TOV_min", "TOV over/under", "TOV over proba", "TOV over", "TOV under",
             "Visitor Link", "Home Link", "Player Link", "Match Link"]]

In [58]:
# Round numbers
# 2 game, over, under
# 3 proba
# 4 min
odds[["MP_game", "3P_game", "3P over", "3P under", "STL_game", "STL over", "STL under", "BLK_game","BLK over", "BLK under", "TOV_game", "TOV over", "TOV under"]] = odds[["MP_game", "3P_game", "3P over", "3P under", "STL_game", "STL over", "STL under", "BLK_game", "BLK over", "BLK under", "TOV_game", "TOV over", "TOV under"]].round(2)
odds[["3P over proba", "3P over", "STL over proba", "BLK over proba", "TOV over proba"]] = odds[["3P over proba", "3P over", "STL over proba", "BLK over proba", "TOV over proba"]].round(3)
odds[["3P_min", "STL_min", "BLK_min", "TOV_min"]] = odds[["3P_min", "STL_min", "BLK_min", "TOV_min"]].round(4)

C:\Users\Uživatel\AppData\Local\Temp\ipykernel_23452\2358733198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  odds[["MP_game", "3P_game", "3P over", "3P under", "STL_game", "STL over", "STL under", "BLK_game","BLK over", "BLK under", "TOV_game", "TOV over", "TOV under"]] = odds[["MP_game", "3P_game", "3P over", "3P under", "STL_game", "STL over", "STL under", "BLK_game", "BLK over", "BLK under", "TOV_game", "TOV over", "TOV under"]].round(2)
C:\Users\Uživatel\AppData\Local\Temp\ipykernel_23452\2358733198.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [59]:
odds.loc[odds["Player"] == "LeBron James"].head(50)

,Date,Visitor,Home,Result,Side,Player,Number,MP_game,3P_game,3P_min,3P over/under,3P over proba,3P over,3P under,STL_game,STL_min,STL over/under,STL over proba,STL over,STL under,BLK_game,BLK_min,BLK over/under,BLK over proba,BLK over,BLK under,TOV_game,TOV_min,TOV over/under,TOV over proba,TOV over,TOV under,Visitor Link,Home Link,Player Link,Match Link
10,"Tue, Oct 24, 2023",Los Angeles Lakers,Denver Nuggets,Home,Visitor,LeBron James,23,35.52,1.77,0.0496,1.5,0.565,1.77,2.30,1.22,0.0344,1.5,0.391,2.56,1.64,0.51,0.0143,0.5,0.496,2.02,1.98,2.90,0.0814,2.5,0.619,1.62,2.63,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/DEN/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202310240DEN.html
667,"Thu, Oct 26, 2023",Phoenix Suns,Los Angeles Lakers,Home,Home,LeBron James,23,34.71,2.32,0.0669,2.5,0.471,2.13,1.89,1.52,0.0440,1.5,0.493,2.03,1.97,0.54,0.0155,0.5,0.592,1.69,2.45,3.24,0.0936,3.5,0.550,1.82,2.22,https://www.basketball-reference.com/teams/PHO/2024.html,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202310260LAL.html
1728,"Sun, Oct 29, 2023",Los Angeles Lakers,Sacramento Kings,Home,Visitor,LeBron James,23,36.75,2.15,0.0587,2.5,0.378,2.65,1.61,1.21,0.0330,1.5,0.327,3.06,1.49,0.54,0.0148,0.5,0.431,2.32,1.76,3.82,0.1038,3.5,0.490,2.04,1.96,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/SAC/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202310290SAC.html
2215,"Mon, Oct 30, 2023",Orlando Magic,Los Angeles Lakers,Home,Home,LeBron James,23,34.64,1.88,0.0541,1.5,0.593,1.69,2.46,1.57,0.0455,1.5,0.458,2.18,1.85,0.62,0.0180,0.5,0.560,1.79,2.27,4.08,0.1181,4.5,0.373,2.68,1.59,https://www.basketball-reference.com/teams/ORL/2024.html,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202310300LAL.html
2906,"Wed, Nov 1, 2023",Los Angeles Clippers,Los Angeles Lakers,Home,Home,LeBron James,23,35.69,2.86,0.0797,2.5,0.402,2.49,1.67,1.06,0.0298,1.5,0.328,3.05,1.49,0.69,0.0192,0.5,0.560,1.78,2.27,3.21,0.0901,3.5,0.469,2.13,1.88,https://www.basketball-reference.com/teams/LAC/2024.html,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202311010LAL.html
3580,"Sat, Nov 4, 2023",Los Angeles Lakers,Orlando Magic,Home,Visitor,LeBron James,23,35.38,1.82,0.0513,1.5,0.638,1.57,2.76,1.64,0.0469,1.5,0.463,2.16,1.86,0.67,0.0190,0.5,0.505,1.98,2.02,4.18,0.1188,4.5,0.358,2.79,1.56,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/ORL/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202311040ORL.html
4238,"Mon, Nov 6, 2023",Los Angeles Lakers,Miami Heat,Home,Visitor,LeBron James,23,36.59,1.66,0.0458,1.5,0.524,1.91,2.10,0.88,0.0243,0.5,0.556,1.80,2.25,0.72,0.0195,0.5,0.416,2.41,1.71,3.68,0.1005,3.5,0.485,2.06,1.94,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/MIA/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketball-reference.com/boxscores/202311060MIA.html
4787,"Wed, Nov 8, 2023",Los Angeles Lakers,Houston Rockets,Home,Visitor,LeBron James,23,35.86,1.88,0.0524,1.5,0.655,1.53,2.90,1.16,0.0325,1.5,0.336,2.97,1.51,0.79,0.0222,0.5,0.598,1.67,2.49,3.18,0.0884,3.5,0.390,2.57,1.64,https://www.basketball-reference.com/teams/LAL/2024.html,https://www.basketball-reference.com/teams/HOU/2024.html,https://www.basketball-reference.com/players/j/jamesle01.html,https://www.basketb

In [60]:
odds.to_csv("odds_final.csv", index = False)

In [61]:
odds.to_excel("odds_final.xlsx", index = False)